## IMPORTS

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
!pip install tqdm
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from google.colab import drive
drive.mount('/content/gdrive')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# Import necessary libraries
from transformers import BertTokenizer, BertModel
import torch

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
trainFolder = "gdrive/My Drive/MLDM - Carrefour Project/data-train"

import os
for dirname, _, filenames in os.walk(f'{trainFolder}'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Mounted at /content/gdrive


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

gdrive/My Drive/MLDM - Carrefour Project/data-train/products_data.csv
gdrive/My Drive/MLDM - Carrefour Project/data-train/test_data.csv
gdrive/My Drive/MLDM - Carrefour Project/data-train/train_data_part_1.csv
gdrive/My Drive/MLDM - Carrefour Project/data-train/train_data_part_10.csv
gdrive/My Drive/MLDM - Carrefour Project/data-train/train_data_part_2.csv
gdrive/My Drive/MLDM - Carrefour Project/data-train/train_data_part_3.csv
gdrive/My Drive/MLDM - Carrefour Project/data-train/train_data_part_4.csv
gdrive/My Drive/MLDM - Carrefour Project/data-train/train_data_part_5.csv
gdrive/My Drive/MLDM - Carrefour Project/data-train/train_data_part_6.csv
gdrive/My Drive/MLDM - Carrefour Project/data-train/train_data_part_7.csv
gdrive/My Drive/MLDM - Carrefour Project/data-train/train_data_part_8.csv
gdrive/My Drive/MLDM - Carrefour Project/data-train/train_data_part_9.csv
gdrive/My Drive/MLDM - Carrefour Project/data-train/products_embe.csv
gdrive/My Drive/MLDM - Carrefour Project/data-train/p

In [2]:
# Import libraries
from tqdm import tqdm
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

### Data Description

This project uses three datasets:

#### `train_data.csv`:
This dataset contains two years (2022 & 2023) of historical transactions for 100,000 Carrefour customers. It has 10 columns:

* ***date***: Date of the transaction
* ***transaction_id***: ID of the transaction
* ***customer_id***: Customer ID
* ***product_id***: Product purchased
* ***has_loyality_card***: Flag indicating whether the customer has a loyalty card
* ***store_id***: Store where the purchase was made
* ***is_promo***: Flag indicating whether there was a discount on the product
* ***quantity***: Quantity purchased of the product
* ***format***: Ecommerce activity format (clcv, lex, or DRIVE)
  - clcv : courses livrées chez vous
  - lex : livraison express
  - DRIVE.
* ***orderChannelCode***: Indicates whether the online activity was made through the website or mobile app

#### `products_data.csv`:
This dataset contains detailed information about the products. The following columns are relevant to this project:

* ***product_id*** : Product name
* ***product_description*** : Product description
* ***department_key***: Department key
* ***class_key***: Class key
* ***subclass_key***: Subclass key
* ***sector***: sector name
* ***brand_key***: Brand name
* ***shelf_level1***: Top-level shelf category
* ***shelf_level2***: Second-level shelf category
* ***shelf_level3***: Third-level shelf category
* ***shelf_level4***: Fourth-level shelf category
* ***sector***: Sector
* ***bio***: Flag indicating whether the product is organic
* ***sugar_free***: Flag indicating whether the product is sugar-free
* ***aspartame_free***: Flag indicating whether the product is aspartame-free
* ***gluten_free***: Flag indicating whether the product is gluten-free
* ***halal***: Flag indicating whether the product is halal
* ***casher***: Flag indicating whether the product is kosher
* ***eco_friendly***: Flag indicating whether the product is eco-friendly
* ***local_french***: Flag indicating whether the product is locally produced in France
* ***artificial_coloring_free***: Flag indicating whether the product is free of artificial coloring
* ***taste_enhancer_free***: Flag indicating whether the product is free of taste enhancers
* ***naturality***: Naturality score
* ***antibiotic_free***: Flag indicating whether the product is antibiotic-free
* ***reduced_sugar***: Flag indicating whether the product has reduced sugar content
* ***vegetarian***: Flag indicating whether the product is vegetarian
* ***pesticide_free***: Flag indicating whether the product is pesticide-free
* ***grain_free***: Flag indicating whether the product is grain-free
* ***no_added_sugar***: Flag indicating whether the product has no added sugar
* ***salt_reduced***: Flag indicating whether the product has reduced salt content
* ***nitrite_free***: Flag indicating whether the product is nitrite-free
* ***fed_without_ogm***: Flag indicating whether the animals were fed without GMOs
* ***no_added_salt***: Flag indicating whether the product has no added salt
* ***no_artificial_flavours***: Flag indicating whether the product has no artificial flavors
* ***porc***: Flag indicating whether the product contains pork
* ***vegan***: Flag indicating whether the product is vegan
* ***frozen***: Flag indicating whether the product is frozen
* ***fat_free***: Flag indicating whether the product is fat-free
* ***reduced_fats***: Flag indicating whether the product has reduced fat content
* ***fresh***: Flag indicating whether the product is fresh
* ***alcool***: Flag indicating whether the product contains alcohol
* ***lactose_free***: Flag indicating whether the product is lactose-free
* ***phenylalanine_free***: Flag indicating whether the product is phenylalanine-free
* ***palm_oil_free***: Flag indicating whether the product is palm oil-free
* ***ecoscore***: Ecoscore
* ***produits_du_monde***: Flag indicating whether the product is an international product
* ***regional_product***: Flag indicating whether the product is a regional product
* ***national_brand***: Flag indicating whether the product is a national brand
* ***first_price_brand***: Flag indicating whether the product is a first-price brand
* ***carrefour_brand***: Flag indicating whether the product is a Carrefour brand

#### `test_data.csv`:
This dataset contains the actual purchases of the first 80,000 customers in 2024. It has three columns:

* ***transaction_id***: ID of the transaction
* ***customer_id***: Customer ID
* ***product_id***: the id of the purchased product

### Load data

* Load *train_data.csv*, *products_data.csv* and *test_data.csv* using pandas.

In [3]:
# This code reads the data from CSV files named "train_data_part_i.csv" for all i from 1 to 10
# and concatenates them into a single pandas DataFrame
train_dataframes = []
for i in tqdm(range(1, 2)): ## 1 - 11
    train_dataframes.append(pd.read_csv(f'{trainFolder}/train_data_part_{i}.csv'))
train_data = pd.concat(train_dataframes, ignore_index=True)

# free up memory by deleting the dataframes we no longer need
del train_dataframes

100%|██████████| 1/1 [00:20<00:00, 20.04s/it]


In [4]:
len(train_data)

8757405

In [5]:
#train_data = train_data.sample(frac=0.1, random_state=1)  # frac=0.1 takes 10% of the data

In [6]:
# This code reads the data from a CSV file named "products_data.csv" into a pandas DataFrame
products_data = pd.read_csv(f'{trainFolder}/products_data.csv')

<ipython-input-6-d8cc84ca7717>:2: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  products_data = pd.read_csv(f'{trainFolder}/products_data.csv')


In [7]:
# This code reads the data from a CSV file named "test_data.csv" into a pandas DataFrame
test_data = pd.read_csv(f'{trainFolder}/test_data.csv')

In [8]:
products_data_emb = pd.read_csv(f'{trainFolder}/products_data_bert1.csv')

In [9]:
import re
import ast

# Function to fix leading spaces and format the embedding
def fix_embedding_format(embedding):
    if not isinstance(embedding, str) or not embedding.strip():
        return None  # Handle missing or null embeddings

    try:
        # Remove leading space, outer brackets, and format properly
        cleaned = embedding.strip()  # Remove leading and trailing spaces
        cleaned = cleaned.lstrip('[')  # Remove the first "[" without spaces
        cleaned = re.sub(r'^\s+', '', cleaned)  # Explicitly remove leading space after '['
        cleaned = cleaned.rstrip(']')  # Remove the final "]"
        formatted = re.sub(r'\s+', ',', cleaned)  # Replace all remaining spaces with commas
        return ast.literal_eval(f'[{formatted}]')  # Convert to a Python list
    except Exception as e:
        print(f"Error fixing embedding: {embedding}, Error: {e}")
        return None  # Return None for rows that fail parsing

# Apply the fix
products_data_emb['embedding'] = products_data_emb['embedding'].apply(fix_embedding_format)

# Verify the fix
print("Sample fixed embedding:", products_data_emb['embedding'].iloc[0])


Sample fixed embedding: [-0.885095, -0.381127149, 0.523640215, -0.206593141, -0.257077605, 0.112018563, -0.29885143, 0.959406734, -0.714935005, 0.129923865, 0.0166194402, -0.0755547956, -0.0377267376, 1.04182911, 0.163626552, 0.751397073, 0.0291998107, 0.551166594, -0.264074653, -0.399605095, -0.201198012, -0.874935985, 1.06481993, 0.0418045074, -0.0420345031, 0.272841483, 0.104067206, -0.250343502, -0.091578193, 0.319404036, -0.209970936, 0.5293383, -0.359875351, -0.21511282, 0.866470695, 0.288113505, 0.590503991, -0.146806508, -0.245605424, 0.263168812, -0.121223584, 0.356522411, 0.157572299, 0.0094260294, 0.379927129, 0.188013539, -3.55864835, -0.106123105, -0.664489448, -0.504262984, 0.303419292, -0.447763115, 0.0806763694, 0.379628479, 0.864170611, 0.570908725, -0.5299896, -0.0537453741, 0.321317106, 0.087104708, 0.538094044, -0.342478484, -0.116726674, 0.121116519, 0.44294259, -0.102612607, 0.337457687, 0.0163220558, -0.519234657, 0.820987165, -0.533936977, -0.325439185, 0.102716

In [10]:
print(products_data_emb['embedding'].head())
print(f"Number of invalid embeddings: {products_data_emb['embedding'].isnull().sum()}")

0    [-0.885095, -0.381127149, 0.523640215, -0.2065...
1    [-0.959080517, -0.365773737, 0.405761927, -0.2...
2    [-0.864090204, -0.386676431, 0.545451403, -0.1...
3    [-0.876440346, -0.245052218, 0.520699978, -0.1...
4    [-0.829412699, -0.389280558, 0.462360829, -0.1...
Name: embedding, dtype: object
Number of invalid embeddings: 0


In [11]:
products_data_emb.head()

,product_id,embedding
0,Product_33508,"[-0.885095, -0.381127149, 0.523640215, -0.2065..."
1,Product_15347,"[-0.959080517, -0.365773737, 0.405761927, -0.2..."
2,Product_80604,"[-0.864090204, -0.386676431, 0.545451403, -0.1..."
3,Product_81740,"[-0.876440346, -0.245052218, 0.520699978, -0.1..."
4,Product_82516,"[-0.829412699, -0.389280558, 0.462360829, -0.1..."


In [12]:
train_data.sample(5)

,date,transaction_id,customer_id,product_id,has_loyality_card,store_id,is_promo,quantity,format,order_channel
3078717,2023-11-16,Transaction_2756092,Household_9590,Product_66161,0,Store_1722,1,2.0,DRIVE,MOBILE_APP
5855212,2022-02-14,Transaction_560443,Household_6971,Product_36780,0,Store_200,1,3.0,DRIVE,MOBILE_APP
3203226,2023-03-30,Transaction_1520773,Household_326,Product_53749,0,Store_197,0,1.0,DRIVE,WEBSITE
7650666,2022-09-21,Transaction_2778575,Household_7573,Product_42748,0,Store_2190,0,2.0,DRIVE,MOBILE_APP
1660038,2022-01-20,Transaction_2213596,Household_4283,Product_61960,0,Store_778,0,2.0,DRIVE,WEBSITE


In [13]:
products_data.sample(3)

,product_id,product_description,department_key,class_key,subclass_key,sector,brand_key,shelf_level1,shelf_level2,shelf_level3,shelf_level4,bio,sugar_free,aspartame_free,gluten_free,halal,casher,eco_friendly,local_french,artificial_coloring_free,taste_enhancer_free,naturality,antibiotic_free,reduced_sugar,vegetarian,pesticide_free,grain_free,no_added_sugar,salt_reduced,nitrite_free,fed_without_ogm,no_added_salt,no_artificial_flavours,porc,vegan,frozen,fat_free,reduced_fats,fresh,alcool,lactose_free,phenylalanine_free,palm_oil_free,ecoscore,produits_du_monde,regional_product,national_brand,first_price_brand,carrefour_brand
39366,Product_13624,4X125G L&F FRAISE 0% MG,Department_25,Class_2513,SubClass_25134,PGC,LIGHT&FREE,Crèmerie et Produits laitiers,Yaourts et Fromages blancs,Yaourts aux fruits et arômatisés,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,NaN,0,0,1,0,0
80081,Product_46717,902704 PYJAMA G BORDEAUX 1314A,Department_62,Class_6223,SubClass_62231,TEXTILE,TEX,Mode et Bagagerie,Garçon,Sous-vêtements et Pyjamas,Pyjamas,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0,0,1
33350,Product_41390,POIRE WILLIAM ROUGE VRAC KG,Department_22,Class_2206,SubClass_22061,PRODUITS FRAIS TRANSFORMATION,ZZZZZZZZZZ,Fruits et Légumes,Fruits,"Pommes, Poires et Raisins",Poires,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,1,0,0


In [14]:
test_data.sample(3)

,transaction_id,customer_id,product_id
660890,Transaction_2024_98094,Household_49910,Product_20411
697373,Transaction_2024_39006,Household_57110,Product_42748
161758,Transaction_2024_39793,Household_11829,Product_61740


## Data Preproccesing

In [15]:
# Ensure 'date' is in datetime format
train_data['date'] = pd.to_datetime(train_data['date'])

# Extract temporal features
train_data['month'] = train_data['date'].dt.month
train_data['day_of_week'] = train_data['date'].dt.dayofweek
train_data['is_weekend'] = train_data['day_of_week'].isin([5, 6]).astype(int)

# Calculate recency: days since the last purchase for each customer
train_data['days_since_last_purchase'] = train_data.groupby('customer_id')['date'].transform(
    lambda x: (x.max() - x).dt.days
)

# Drop the 'date' column if it's no longer needed
train_data = train_data.drop(columns=['date'], errors='ignore')

In [16]:
# Aggregate customer-specific features
customer_features = train_data.groupby('customer_id').agg(
    purchase_frequency=('product_id', 'count'),  # Total number of purchases
    avg_quantity=('quantity', 'mean'),          # Average quantity purchased
    promo_ratio=('is_promo', 'mean'),           # Proportion of purchases made with promotions
    unique_products=('product_id', 'nunique')   # Number of unique products purchased
).reset_index()

# Merge these features into train_data and test_data
train_data = train_data.merge(customer_features, on='customer_id', how='left')

In [17]:
# Check for missing values
print("Missing values in train_data:")
print(train_data.isnull().sum())

print("Missing values in test_data:")
print(test_data.isnull().sum())

# Fill remaining missing values with zeros as a safety measure
train_data = train_data.fillna(0)
test_data = test_data.fillna(0)

Missing values in train_data:
transaction_id              0
customer_id                 0
product_id                  0
has_loyality_card           0
store_id                    0
is_promo                    0
quantity                    0
format                      0
order_channel               0
month                       0
day_of_week                 0
is_weekend                  0
days_since_last_purchase    0
purchase_frequency          0
avg_quantity                0
promo_ratio                 0
unique_products             0
dtype: int64
Missing values in test_data:
transaction_id    0
customer_id       0
product_id        0
dtype: int64


In [18]:
train_data.head()

,transaction_id,customer_id,product_id,has_loyality_card,store_id,is_promo,quantity,format,order_channel,month,day_of_week,is_weekend,days_since_last_purchase,purchase_frequency,avg_quantity,promo_ratio,unique_products
0,Transaction_1730125,Household_39,Product_5362,0,Store_2,0,1.0,DRIVE,MOBILE_APP,11,2,0,0,282,1.219858,0.014184,185
1,Transaction_1560535,Household_39,Product_67174,0,Store_2,0,2.0,DRIVE,WEBSITE,7,2,0,483,282,1.219858,0.014184,185
2,Transaction_1560535,Household_39,Product_82254,0,Store_2,0,2.0,DRIVE,WEBSITE,7,2,0,483,282,1.219858,0.014184,185
3,Transaction_1730125,Household_39,Product_3895,0,Store_2,0,1.0,DRIVE,MOBILE_APP,11,2,0,0,282,1.219858,0.014184,185
4,Transaction_1560535,Household_39,Product_34014,0,Store_2,0,1.0,DRIVE,WEBSITE,7,2,0,483,282,1.219858,0.014184,185


In [19]:
train_data = train_data.drop(columns=['has_loyality_card', 'store_id', 'day_of_week','is_weekend' ,'format','order_channel','month'])

In [20]:
train_data.head()

,transaction_id,customer_id,product_id,is_promo,quantity,days_since_last_purchase,purchase_frequency,avg_quantity,promo_ratio,unique_products
0,Transaction_1730125,Household_39,Product_5362,0,1.0,0,282,1.219858,0.014184,185
1,Transaction_1560535,Household_39,Product_67174,0,2.0,483,282,1.219858,0.014184,185
2,Transaction_1560535,Household_39,Product_82254,0,2.0,483,282,1.219858,0.014184,185
3,Transaction_1730125,Household_39,Product_3895,0,1.0,0,282,1.219858,0.014184,185
4,Transaction_1560535,Household_39,Product_34014,0,1.0,483,282,1.219858,0.014184,185


## BERT

In [21]:
print("GPU available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")

GPU available: False
GPU name: No GPU detected


In [22]:
!pip install --upgrade torch torchvision torchaudio
!pip install lightgbm --install-option=--gpu


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: --install-option


In [23]:
import pandas as pd
import numpy as np
import ast


def safe_literal_eval(x):
    if isinstance(x, list):  # Check if the input is already a list
        return x  # If it's a list, return it directly

    # Otherwise, try to parse as string
    try:
        return ast.literal_eval(x)
    except (SyntaxError, ValueError):
        # If literal_eval fails, try to interpret as NumPy array
        try:
            return np.fromstring(x[1:-1], sep=',').tolist()  # Handle array-like strings
        except ValueError:
            # If all attempts fail, return the original value or handle it differently
            return x

products_data_emb['embedding'] = products_data_emb['embedding'].apply(safe_literal_eval)

In [24]:
# Merge product embeddings into train_data and test_data
train_data = train_data.merge(products_data_emb, on='product_id', how='left')
test_data = test_data.merge(products_data_emb, on='product_id', how='left')

In [25]:
train_data.head()

,transaction_id,customer_id,product_id,is_promo,quantity,days_since_last_purchase,purchase_frequency,avg_quantity,promo_ratio,unique_products,embedding
0,Transaction_1730125,Household_39,Product_5362,0,1.0,0,282,1.219858,0.014184,185,"[-0.714311779, -0.163191468, 0.362559706, -0.2..."
1,Transaction_1560535,Household_39,Product_67174,0,2.0,483,282,1.219858,0.014184,185,"[-0.798357368, -0.396710098, 0.51669389, -0.09..."
2,Transaction_1560535,Household_39,Product_82254,0,2.0,483,282,1.219858,0.014184,185,"[-0.704730511, -0.268889457, 0.566852391, -0.0..."
3,Transaction_1730125,Household_39,Product_3895,0,1.0,0,282,1.219858,0.014184,185,"[-0.843781114, -0.310565889, 0.639390349, -0.2..."
4,Transaction_1560535,Household_39,Product_34014,0,1.0,483,282,1.219858,0.014184,185,"[-0.797278345, -0.273620039, 0.310368985, -0.2..."


In [26]:
test_data.head()

,transaction_id,customer_id,product_id,embedding
0,Transaction_2024_1,Household_16874,Product_9790,"[-0.880482972, -0.261171252, 0.597862303, 0.00..."
1,Transaction_2024_1,Household_16874,Product_68295,"[-0.858973503, -0.308066607, 0.0728522763, -0...."
2,Transaction_2024_1,Household_16874,Product_19494,"[-0.949451506, -0.309663713, 0.494894505, -0.0..."
3,Transaction_2024_1,Household_16874,Product_11109,"[-0.781185567, -0.0642214194, 0.0687608123, -0..."
4,Transaction_2024_4,Household_9247,Product_57151,"[-0.919838369, -0.272809863, 0.346421629, -0.0..."


### Prepare Data for Neural Network

In [27]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np

In [28]:
from tqdm import tqdm
tqdm.pandas()  # Initialize tqdm for pandas

# Convert embeddings to a list of tensors during data loading, with progress bar
#train_data['embedding'] = train_data['embedding'].progress_apply(lambda x: torch.tensor(x, dtype=torch.float32))
#test_data['embedding'] = test_data['embedding'].progress_apply(lambda x: torch.tensor(x, dtype=torch.float32))

In [29]:
class CarrefourDataset(Dataset):
    def __init__(self, data, num_negatives=4):
        self.data = data
        self.num_negatives = num_negatives
        self.customer_ids = data['customer_id'].values
        self.product_ids = data['product_id'].values
        self.features = data[['is_promo', 'quantity', 'days_since_last_purchase', 'purchase_frequency',
                             'avg_quantity', 'promo_ratio', 'unique_products']].values
        self.embeddings = data['embedding'].values
        self.create_training_data()

    def create_training_data(self):
        self.training_data = []
        product_set = set(self.product_ids)
        negative_samples = np.random.choice(list(product_set), size=len(self.customer_ids) * self.num_negatives, replace=True)
        negative_idx = 0

        for idx in tqdm(range(len(self.customer_ids)), desc="Creating training data"):
            customer_id = self.customer_ids[idx]
            product_id = self.product_ids[idx]
            features = self.features[idx]
            embedding = self.embeddings[idx]
            # Positive sample
            self.training_data.append((customer_id, product_id, features, embedding, 1))
            # Negative samples
            for _ in range(self.num_negatives):
                negative_product = negative_samples[negative_idx]
                negative_idx += 1
                self.training_data.append((customer_id, negative_product, features, embedding, 0))

    def __len__(self):
        return len(self.training_data)

    def __getitem__(self, idx):
        customer_id, product_id, features, embedding, label = self.training_data[idx]
        features = torch.tensor(features, dtype=torch.float32)
        embedding = torch.tensor(embedding, dtype=torch.float32)  # Ensure embedding is converted to a tensor
        label = torch.tensor([label], dtype=torch.float32)  # Change to tensor with a list to prevent ValueError
        return customer_id, product_id, features, embedding, label

print("Creating training dataset with progress...")
train_dataset = CarrefourDataset(train_data)
print("Training dataset created.")
print("Creating DataLoader for training...")
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
print("DataLoader for training created.")


Creating training dataset with progress...


Creating training data: 100%|██████████| 8762320/8762320 [01:31<00:00, 96101.92it/s] 


Training dataset created.
Creating DataLoader for training...
DataLoader for training created.


### Define and Train Neural Network

In [30]:
# Step 3: Define the Neural Network Model
class RecommendationModel(nn.Module):
    def __init__(self, feature_size, embedding_size):
        super(RecommendationModel, self).__init__()
        # Updated layers with more units and additional layers
        self.fc1 = nn.Linear(feature_size + embedding_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.output = nn.Linear(64, 1)

        # Activation functions and normalization layers
        self.leaky_relu = nn.LeakyReLU(0.01)
        self.batch_norm1 = nn.BatchNorm1d(256)
        self.batch_norm2 = nn.BatchNorm1d(128)
        self.dropout = nn.Dropout(0.3)

    def forward(self, features, embedding):
        # Concatenate features and embedding
        if isinstance(embedding, list):
            embedding = torch.tensor(embedding, dtype=torch.float32)
        x = torch.cat((features, embedding), dim=1)

        # Forward pass through the updated layers
        x = self.batch_norm1(self.leaky_relu(self.fc1(x)))
        x = self.dropout(x)
        x = self.batch_norm2(self.leaky_relu(self.fc2(x)))
        x = self.dropout(x)
        x = self.leaky_relu(self.fc3(x))

        # Output without sigmoid (BCEWithLogitsLoss handles sigmoid internally)
        x = self.output(x)
        return x

# Feature size is 7 (excluding the embedding), embedding size depends on BERT output
test_embedding = train_data['embedding'][0]
embedding_size = len(test_embedding)
feature_size = 7

model = RecommendationModel(feature_size, embedding_size)



### Training

In [31]:
# Step 4: Define Loss Function and Optimizer
criterion = nn.BCEWithLogitsLoss() # Binary Cross-Entropy Loss for classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Step 5: Training Loop
epochs = 1
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for _, _, features, embedding, label in tqdm(train_loader, desc=f'Epoch {epoch+1} training', leave=False):
        optimizer.zero_grad()
        outputs = model(features, embedding).view(-1)
        loss = criterion(outputs, label.view(-1))  # Ensure label is reshaped to match output size
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')


# Step 6: Save the Model
torch.save(model.state_dict(), 'recommendation_model.pth')


Epoch 1, Loss: 0.5005179311396495


### TEST

In [ ]:
# Step 7: Evaluation on Test Data (Hit Rate @10 and Accuracy)
class TestDataset(Dataset):
    def __init__(self, data):
        self.customer_ids = data['customer_id'].values
        self.product_ids = data['product_id'].values
        self.embeddings = data['embedding'].values

    def __len__(self):
        return len(self.customer_ids)

    def __getitem__(self, idx):
        customer_id = self.customer_ids[idx]
        product_id = self.product_ids[idx]
        embedding = torch.tensor(self.embeddings[idx], dtype=torch.float32)
        return customer_id, product_id, embedding

test_dataset = TestDataset(test_data)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

model.eval()
hit_count = 0
total_count = 0
correct_predictions = 0
all_predictions = 0

with torch.no_grad():
    # Group by customer_id to generate recommendations for each customer
    customers = test_data['customer_id'].unique()

    for customer_id in tqdm(customers, desc='Evaluating Test Data'):
        # Get all products for the given customer
        customer_data = test_data[test_data['customer_id'] == customer_id]
        product_ids = customer_data['product_id'].values
        embeddings = torch.stack([torch.tensor(emb, dtype=torch.float32) for emb in customer_data['embedding'].values])

        # Create a placeholder for features (since we don't have features in the test set)
        features = torch.zeros(embeddings.shape[0], feature_size)

        # Get model scores for all products for this customer
        outputs = model(features, embeddings).view(-1)

        # Get the actual product purchased by this customer
        actual_product_id = product_ids[0]  # Assuming there's only one transaction in test data per customer

        # Get the number of products for this customer
        num_products = len(product_ids)

        # Determine the value of k (top-N)
        k = min(10, num_products)  # Ensure k does not exceed the number of available products

        # Get top-k recommended product indices
        top_k_indices = torch.topk(outputs, k=k).indices
        top_k_product_ids = product_ids[top_k_indices.numpy()]

        # Check if the actual product is in the top-k recommendations
        if actual_product_id in top_k_product_ids:
            hit_count += 1
        total_count += 1

        # Accuracy calculation: how many of the top-k predictions are correct
        predicted_labels = (outputs > 0.5).float()
        true_labels = torch.zeros_like(predicted_labels)
        true_labels[0] = 1  # Assuming the first product in customer_data is the correct one

        correct_predictions += torch.sum(predicted_labels == true_labels).item()
        all_predictions += len(predicted_labels)

# Avoiding division by zero
hit_rate = hit_count / total_count if total_count > 0 else 0
accuracy = correct_predictions / all_predictions if all_predictions > 0 else 0

print(f'Hit Rate @10: {hit_rate:.4f}')
print(f'Accuracy: {accuracy:.4f}')


Evaluating Test Data: 100%|██████████| 80000/80000 [2:15:41<00:00,  9.83it/s]

Hit Rate @10: 0.7068
Accuracy: 0.9345


# Create submission file

The goal of this part is to provide a function that allows you to encode your prediction in a format that is readable by kaggle when you submit it. In particular, this function checks that you have 10 distinct products per customer and that the ranks are some distinct integers between 1 to 10.

In [ ]:
import pandas as pd
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

# Assuming you have the trained model and test data ready
# Load the trained model
model = RecommendationModel(feature_size, embedding_size)
model.load_state_dict(torch.load('recommendation_model.pth'))
model.eval()

# Assuming 'test_dataset' is already created from the previous code
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Generate predictions
predictions = []

with torch.no_grad():
    for customer_id, product_id, embedding in tqdm(test_loader, desc='Generating Predictions'):
        # Create a placeholder for features (since we don't have features in the test set)
        features = torch.zeros(embedding.shape[0], feature_size)
        outputs = model(features, embedding).view(-1)

        # Get top 10 recommended product indices
        top_k_indices = torch.topk(outputs, k=10).indices
        top_k_product_ids = [product_id[idx].replace('Product_', '') for idx in top_k_indices]  # Remove 'Product_' prefix

        # Append predictions
        for i in range(len(customer_id)):
            predictions.append({
                'customer_id': customer_id[i],  # Removed .item() to handle strings correctly
                'product_id': top_k_product_ids
            })

# Create DataFrame from predictions
predictions_df = pd.DataFrame(predictions)

# Create 'rank' column
predictions_df['rank'] = predictions_df['product_id'].apply(lambda x: list(range(1, len(x) + 1)))

# Convert lists to comma-separated strings
predictions_df['product_id'] = predictions_df['product_id'].apply(lambda x: ','.join(map(str, x)))
predictions_df['rank'] = predictions_df['rank'].apply(lambda x: ','.join(map(str, x)))

# Remove any letters or underscores from 'customer_id' and 'product_id'
predictions_df['customer_id'] = predictions_df['customer_id'].astype(str).str.replace(r'[^0-9]', '', regex=True)
predictions_df['product_id'] = predictions_df['product_id'].str.replace(r'[^0-9,]', '', regex=True)

predictions_df.head()


In [144]:
predictions_df.to_csv(f'gdrive/My Drive/MLDM - Carrefour Project/submission/submission_01.csv', index=False)